# Step 1: Import all Required Libraries

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Step 2: Load and preprocess the data

In [2]:
# Define paths to training and validation data
train_dir = r'D:\Deep Learning\Text_Image_Language_Classifier\Text Image Dataset\train'
val_dir = r'D:\Deep Learning\Text_Image_Language_Classifier\Text Image Dataset\test'


In [3]:
# Image data generator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255.,  # Rescale pixel values from 0-255 to 0-1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
# Validation data generator should only rescale the images
val_datagen = ImageDataGenerator(rescale=1./255.)

In [5]:
# Create generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to match the input size of the model
    batch_size=32,
    class_mode='categorical'  # Use categorical labels
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # Resize images to match the input size of the model
    batch_size=32,
    class_mode='categorical'  # Use categorical labels
)

Found 10575 images belonging to 12 classes.
Found 3452 images belonging to 12 classes.


# Step 3: Load the ResNet50 model without the top layer

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
# Freeze the layers of ResNet50
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

In [10]:
model.compile(optimizer=SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [12]:
resnet_history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


c:\Users\mohan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


331/331 ━━━━━━━━━━━━━━━━━━━━ 883s 3s/step - accuracy: 0.2261 - loss: 2.3216 - val_accuracy: 0.4432 - val_loss: 1.9787
Epoch 2/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 875s 3s/step - accuracy: 0.3371 - loss: 1.9590 - val_accuracy: 0.3960 - val_loss: 1.8554
Epoch 3/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 1381s 4s/step - accuracy: 0.3770 - loss: 1.8062 - val_accuracy: 0.4322 - val_loss: 1.8060
Epoch 4/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 2353s 7s/step - accuracy: 0.3995 - loss: 1.7137 - val_accuracy: 0.4392 - val_loss: 1.8062
Epoch 5/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 1233s 4s/step - accuracy: 0.4300 - loss: 1.6535 - val_accuracy: 0.4661 - val_loss: 1.7877
Epoch 6/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 876s 3s/step - accuracy: 0.4420 - loss: 1.5875 - val_accuracy: 0.4429 - val_loss: 1.7467
Epoch 7/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 860s 3s/step - accuracy: 0.4649 - loss: 1.5412 - val_accuracy: 0.4577 - val_loss: 1.7454
Epoch 8/10
331/331 ━━━━━━━━━━━━━━━━━━━━ 872s 3s/step - accuracy: 0.4839 - loss: 1.4871 - val_accuracy: 0.

In [13]:
test_loss, test_acc = model.evaluate(val_generator, verbose=2)
print('Validation accuracy:', test_acc)

108/108 - 188s - 2s/step - accuracy: 0.4783 - loss: 1.6976
Validation accuracy: 0.4782734513282776
